# Capacitação Vialab # Atividade 11

## Aprender sobre os difentes tipos de rede recorrentes
### Data de atualização: 08/02/2022

# Objetivo: Aprender conceitos e aplicações básicas de diferentes tipos de redes recorrentes

# Predição de séries temporais utilizando redes neurais recorrentes simples,  LSTM e GRU


Aqui desenvolveremos duas redes neurais recorrentes, dos tipos simples, LSTM e GRU, utilizando o Keras, para demonstrar sua capacidade em prever séries temporais


In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import keras
from tensorflow.keras import backend as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


Para garantir que nossos resultados sejam reprodutíveis, vamos fixar a semente de aleatorização (random seed) 

In [ ]:
# fixa random seed para garantir reprodutibilidade
np.random.seed(0)

## Carregando os dados

O dataset a ser utilizado é a produção de $CO_2$ em partes por milhão (ppm) do vulcão Mauna Loa
(https://en.wikipedia.org/wiki/Mauna_Loa) entre 1965-1980 por mês. 
Baixe o CSV e mova ele para a mesma pasta deste notebook: https://github.com/gahjelle/data-analysis-with-python/blob/7305e2eecb6f3356539128703ad5d61e04a047c3/data/co2-ppm-mauna-loa-19651980.csv

Os dados serão carregados utilizando o Pandas. Não estamos interessados no campo "data", uma vez que cada observação está separada pelo mesmo intervalo de um mês. Assim, ao carregarmos os dados podemos excluir a primeira coluna. 

O arquivo CSV possui a informação no rodapé que pode ser também excluída (argumento pandas.read_csv() configurado para 3 para as últimas 3 linhas). 

In [ ]:
# carrega o dataset
df = pd.read_csv('./co2-ppm-mauna-loa-19651980.csv', usecols=[1], engine='python', skipfooter=3)
data = df.values
data = data.astype('float32')
print('Quantidade de dados no arquivo:',len(data))

## Visualizando os dados

In [ ]:
plt.figure(figsize=[10,5])
plt.xlabel('TimePoint in Months')
plt.ylabel('$CO_2$(ppm)')
plt.grid()
plt.plot(data);

Em séries temporais, a sequência dos dados é importante. Para criar a divisão entre treinamento e teste, utilizaremos a primeira parte da série para treinamento, e a última para o teste. 


In [ ]:
# split into train and test sets
train_size = int(len(data) * 0.7)
train, test = data[0:train_size,:], data[train_size:len(data),:]
print('Dataset de treinamento contém: ', len(train),' dados')
print('Dataset de teste contém: ', len(test),' dados')

Os LSTMs são sensíveis à escala dos dados de entrada, especificamente quando as funções de ativação sigmóide ou tanh são usadas. É fundamental normalizar os dados para o intervalo de [0, 1]. Isso pode ser feito usando a classe de pré-processamento MinMaxScaler da biblioteca scikit-learn.

### Problema 1

Use o MinMaxScaler para normalizar o conjunto de treinamento e testes entre 0 e 1. 

# Por que normalizar os dados?

A normalização é um estágio de pré-processamento de dados. A normalização é um processo importante para a manipulação de dados, sendo assim, há remodelagem dos dados para uma escala numérica padrão. O processo que podemos é a escalabilidade de dados, podendo aumentá-la ou reduzi-la, antes de ser usado em outros estágios. A normalização é necessária quando os dados se encontram em diferentes escalas, caso contrário, pode levar a execuções ruins do modelo. Dito isso, há várias técnicas para realizar a normalização de dados.  

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data_train = scaler.fit_transform(data)
norm_train = scaler.transform(train)
norm_test = scaler.transform(test)

In [ ]:
plt.figure(figsize=[10,5])
plt.xlabel('Meses')
plt.ylabel('$CO_2$(ppm) normalizado')
plt.grid()
plt.plot(range(len(train)),norm_train,'b')
plt.plot(range(len(train),len(train)+len(test)),norm_test,'r')
plt.legend(['Treinamento','Teste']);

### Problema 2

Podemos escrever uma função que separa os dados em um array de entrada (X) que tenha todo o dado no tempo t-i, e um array de saída que contenha os dados no momento t.

A função usa dois argumentos: os dados e o look_back, que é o número de timesteps anteriores a serem usados como variáveis de entrada para prever o próximo período de tempo - o padrão é 1

In [ ]:
def split_X_y(data, look_back = 1, look_ahead=1):
    X, y = [], []
    for i in range(len(data)-look_back-look_ahead):
        val = data[i:i+look_back,0]
        X.append(val)
        y.append(data[i+look_back:i+look_back+look_ahead,0])
    return np.array(X), np.array(y)

In [ ]:
# reshape em X contendo os dados para as amostras t-i ... t-2 t-1 e Y contendo as amostras t
look_back = 1 # olha somente para a amostra anterior para prever a próxima amostra


A RNN simples espera que os dados de entrada (X) sejam fornecidos na forma de: [sample, time steps, features].

Atualmente, os dados estão no formato: [samples, features], e estamos modelando o problema como um timestep para cada amostra. Usando numpy.reshape () da seguinte forma fazemos a devida transformação:

In [ ]:
# # reshape a entrada para [samples, time steps, features]
def reshape_train_test(look_back, split_size):
    trainX_whole, trainY_whole = split_X_y(norm_train, look_back)
    testX, testY = split_X_y(norm_test, look_back)
    trainX_whole = np.reshape(trainX_whole, (trainX_whole.shape[0], look_back, data.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], look_back, data.shape[1]))

    # cria o dataset de validação
    trainX, valX, trainY, valY = train_test_split(trainX_whole, trainY_whole, test_size=split_size)
    return trainX_whole, trainX, valX, testX, trainY_whole, trainY, valY, testY

In [ ]:
trainX_whole, trainX, valX, testX, trainY_whole, trainY, valY, testY = reshape_train_test(look_back, 0.7)

print('Shape de x_train', trainX.shape)
print('Shape de x_val', valX.shape)
print('Shape de x_test', testX.shape)

print('Shape de y_train', trainY.shape)
print('Shape de y_val', valY.shape)
print('Shape de y_test', testY.shape)

Vamos agora projetar e ajustar nossa RNN simples.

A rede tem uma camada visível com 1 entrada, uma camada oculta com 4 blocos recorrentes ou neurônios e uma camada de saída que faz uma previsão de valor único. A função de ativação sigmóide padrão é usada para os blocos recorrentes. A rede é treinada por 20 épocas e um tamanho de lote de 1 é usado.

### Rede RNN simples

In [ ]:
# cria e ajusta a RNN simples 
K.clear_session()
model = Sequential()
model.add(SimpleRNN(4, input_shape=(look_back,data.shape[1])))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(trainX, 
                    trainY, 
                    epochs=20, 
                    batch_size=1, 
                    verbose=1, 
                    validation_data=(valX, valY))


Vamos agora definir uma função para fazer as previsões e plotar. 
Uma vez que o modelo estiver ajustado, podemos estimar o desempenho do modelo nos conjuntos de dados de treinamento e teste.

Observe que devemos inverter (desnormalizar) as previsões antes de calcular a acurácia para garantir que o desempenho seja comparado nas mesmas unidades que os dados originais (ppm por mês).

As previsões foram geradas usando o modelo para o conjunto de dados de treinamento e de teste. Também podemos visualizar os resultados para ter uma indicação de como o modelo funciona.

Ao plotar os dados, devemos deslocar as previsões pelo look_back no tempo para alinhar no eixo x com o conjunto de dados original. Os dados são apresentados com o conjunto de dados original como pontos pretos, as previsões para o conjunto de dados de treinamento em azul e as previsões no conjunto de dados de teste em vermelho.

In [ ]:
def plot_history_predictions(history, Xtrain, Ytrain, Xtest, Ytest, scaler, model, title, xlabel, ylabel, lookback):
    # Resumo do historico de loss
    plt.figure(figsize=(20, 5))
    plt.plot(history.history['loss'], color='blue')
    plt.plot(history.history['val_loss'], color='red')
    plt.title('Model loss', fontsize=20)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper right', fontsize=14)
    plt.show()

    # faz as predições
    trainPredict = model.predict(Xtrain)
    testPredict = model.predict(Xtest)

    # inverte as predições
    trainPredict = scaler.inverse_transform(trainPredict)
    trainYTrue = scaler.inverse_transform(Ytrain)
    testPredict = scaler.inverse_transform(testPredict)
    testYTrue = scaler.inverse_transform(Ytest)

    # calcula o root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainYTrue, trainPredict))
    print('Treinamento: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testYTrue, testPredict))
    print('Teste: %.2f RMSE' % (testScore))

    plt.figure(figsize=[10,5])  
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.plot(range(len(data)),data,'k.')
    plt.plot(range(look_back,len(trainPredict)+look_back),trainPredict,'b')
    plt.plot(range(len(trainPredict)+2*look_back,len(trainPredict)+len(testPredict)+2*look_back),testPredict,'r')
    plt.legend(['Original','Treinamento','Teste']);
    

In [ ]:
plot_history_predictions(history, trainX_whole, trainY_whole, testX, testY, scaler, model, 
                         'Previsão com RNN simples', 'Meses', '$CO_2$(ppm)', look_back)

### Rede LSTM

Vamos agora projetar e ajustar nossa rede LSTM.


In [ ]:
# cria e ajusta a rede LSTM 
modelLSTM = Sequential()
modelLSTM.add(LSTM(4, input_shape=(look_back, data.shape[1])))
modelLSTM.add(Dense(1))

modelLSTM.compile(loss='mean_squared_error', optimizer='adam')

history = modelLSTM.fit(trainX, 
                    trainY, 
                    epochs=20, 
                    batch_size=1, 
                    verbose=1, 
                    validation_data=(valX, valY))



In [ ]:
plot_history_predictions(history, trainX_whole, trainY_whole, testX, testY, scaler, modelLSTM, 
                         'Previsão com LSTM', 'Meses', '$CO_2$(ppm)', look_back)


### Problema 3

Melhore os resultados acima tentando o seguinte:
- Aumentar o número de épocas
- Aumentar o valor look_back
- Modificar a arquitetura
- Outros...

Como exemplo, use look_back de 5, aumente o tamanho do dataset de treinamento, aumente o número de épocas e adicione uma camada de dropout


In [ ]:
look_back = 5 # olha somente para as amostras anteriores para prever a próxima amostra
trainX_whole, trainX, valX, testX, trainY_whole, trainY, valY, testY = reshape_train_test(look_back, 0.7)


In [ ]:
del modelLSTM
modelLSTM = Sequential()
modelLSTM.add(LSTM(4, input_shape=(look_back,data.shape[1])))
modelLSTM.add(Dropout(0.2))
modelLSTM.add(Dense(1))
modelLSTM.compile(loss='mean_squared_error', optimizer='adam')
history = modelLSTM.fit(trainX, 
                    trainY, 
                    epochs=20, 
                    batch_size=1, 
                    verbose=1, 
                    validation_data=(valX, valY))


In [ ]:
plot_history_predictions(history, trainX_whole, trainY_whole, testX, testY, scaler, modelLSTM, 
                         'Previsão com LSTM', 'Meses', '$CO_2$(ppm)', look_back)


### Problema 4 - Rede GRU

Utilizar outro modelo, a GRU


In [ ]:
# reshape em X contendo os dados para as amostras t-i ... t-2 t-1 e Y contendo as amostras t
look_back = 5 # olha somente para as amostras anteriores para prever a próxima amostra
trainX_whole, trainX, valX, testX, trainY_whole, trainY, valY, testY = reshape_train_test(look_back, 0.7)

In [ ]:
modelGRU = Sequential()
modelGRU.add(GRU(4, input_shape=(look_back, data.shape[1])))
modelGRU.add(Dropout(0.2))
modelGRU.add(Dense(1))
modelGRU.compile(loss='mean_squared_error', optimizer='adam')
history = modelGRU.fit(trainX, 
                    trainY, 
                    epochs=20, 
                    batch_size=1, 
                    verbose=1, 
                    validation_data=(valX, valY))



In [ ]:
plot_history_predictions(history, trainX_whole, trainY_whole, testX, testY, scaler, modelGRU, 
                         'Previsão com GRU', 'Meses', '$CO_2$(ppm)', look_back)


### Problema 5 - Rede CNN1D

Utilizar outro modelo, a CNN1D


In [ ]:
modelCNN1D = Sequential()
modelCNN1D.add(Conv1D(filters=64, kernel_size=1, activation='tanh', input_shape=(look_back, data.shape[1])))
modelCNN1D.add(Conv1D(filters=64, kernel_size=1, activation='tanh'))
modelCNN1D.add(MaxPooling1D(pool_size=1))
modelCNN1D.add(Dropout(0.2))
modelCNN1D.add(Flatten())
modelCNN1D.add(Dense(10, activation='tanh'))
modelCNN1D.add(Dense(1, activation='linear'))

modelCNN1D.summary()

In [ ]:
tx = 0.001
verbose = 1
epochs = 20
batch_size = 32

perda = 'mae'
metrica = 'mse'

modelCNN1D.compile(loss=[perda], optimizer=tf.keras.optimizers.Adam(lr = tx), metrics=[metrica])

history = modelCNN1D.fit(trainX, trainY, epochs=epochs, verbose=verbose, validation_data=(valX, valY), batch_size=batch_size)

In [ ]:
plot_history_predictions(history, trainX_whole, trainY_whole, testX, testY, scaler, modelCNN1D, 
                         'Previsão com CNN1D', 'Meses', '$CO_2$(ppm)', look_back)


### Exercício

Neste exercício, vamos construir um preditor de séries temporais e treiná-lo para prever uma única série temporal. Usaremos um conjunto de dados fornecido pelo [UCI Machine Learning Repository] (https://archive.ics.uci.edu/ml/datasets/PM2.5+Data+of+Five+Chinese+Cities) que possui dados de monitoramento da qualidade do ar nas cidades chinesas / distritos 1. Os dados se referem a concentração de material particulado de diametro menor que 2,5 micrometros (PM2.5), que são as partículas finas inaláveis pelo ser humano e perigosas a saúde. O período de monitoramento é de 01/01/2010 a 31/12/2015. Dados faltantes são denotados como NaN.

Liang, X., S. Li, S. Zhang, H. Huang, and S. X. Chen (2016), PM2.5 data reliability, consistency, and air quality assessment in five Chinese cities, J. Geophys. Res. Atmos., 121, 10220 to 10236, [Web Link].


#### Configurando os dados

Começaremos trabalhando com dados de Pequim e filtraremos o conjunto de dados para registros a partir de 2015.




### A atividade é prever para as próximas 6h o valor da concentração de PM2.5. 

Utilize os conceitos aprendidos até aqui, testando diferentes modelos de IA e diferentes abordagens para conseguir prever com o melhor desempenho o valor das próximas 6h da concentração de PM2.5 para uma série histórica de sua escolha.

1. Inicialmente, escolha uma estação de monitoramento de uma das cidades com que irá trabalhar, e faça uma análise exploratória dos dados, avaliando estatisticamente e graficamente como os dados se comportam.
2. Explore as técnicas MLP, RNN simples, LSTM, GRU, CNN1D, FCN+LSTM e Conv2DLSTM, utilizando a camada TimeDistributed;
3. Escolha, com base na avaliação do loss e das métricas MAE, MSE, NMSE, r, R2 e Fac2 qual foi o melhor modelo;
4. Use o ano de 2015 para fins de teste;
5. Apresente os resultados de forma lógica, organizada e que seja reprodutível pelo professor;


Importante: o trabalho deve ser feito em grupo. <br>

Importante2: Conseguir utilizar as informações meteorológicas para melhorar a qualidade do modelo final;  <br>
Importante3: Utilização de wavelets para feature augmentation. <br>



## Download e preparação do arquivo:
- Baixe o zip do link: https://archive.ics.uci.edu/ml/machine-learning-databases/00394/FiveCitiePMData.rar
- Faça a extração do arquivo BeijingPM... e renomeie ele para Beijing.csv
- Mova o arquivo para uma pasta no seu drive
- Execute a próxima célula e autorize o acesso do Colab ao seu drive
- insira o caminho do seu drive até o arquivo Beijing.csv, exemplo: 

`df_Beijing = pd.read_csv('/content/drive/MyDrive/IC/Beijing.csv')`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_Beijing = pd.read_csv('./sample_data/Beijing.csv')
df_Beijing = df_Beijing[df_Beijing.year >= 2015]
df_Beijing.head(10)

In [ ]:
# interpolando os dados
df_Beijing['PM_Dongsi'] = df_Beijing['PM_Dongsi'].interpolate()
df_Beijing['PM_Dongsi'].head(10)